# Introduction to physics informed neural networks (PINNS)
In the previous examples we have trained neural networks to represent function by feeding it with a set of observations at certain input points $x, y(x)$. and we have minimized the mean squared error between observations and predictions:
This in turn may require many training samples, particularly in the case of complex functions. In the case one has prior information regarding the physical behaviour of $y$ such information could be used in combination with observations when training a neural network. In this setting recent developments in [automatic differentiation](https://arxiv.org/abs/1502.05767) can be exploited, which allow to differentiate neural networks with respect to their input coordinates to obtain [physics informed neural networks](https://arxiv.org/abs/1711.10561) (PINNS). In this setting let us consider a general nonlinear partial differential equation of the general form

\begin{equation}
u_t +  \Lambda \left[u;\lambda \right] = 0\,,
\end{equation}

where $\Lambda$ is a differential operator, and $\lambda$ are parameters. In such a case are obtained training a neural network, such that it minimize discenpencies between observations $u\left(x_i, t^n \right)$ and predictions, while also satisfying (or at least minimize deviation from) the left hand side of it's govering nequation:
\begin{equation}
f := u_t +  \Lambda \left[ u \right] \,,
\end{equation}

In order to obtain a loss function
\begin{equation}
loss  = \mathrm{MSE}_u + \mathrm{MSE}_f = \frac{1}{N_u}\sum_{i=1}^{N_u}\left(u\left(t_u^i, x_u^i\right) - u_i\right)^2 + \frac{1}{N_f}\sum_{i=1}^{N_f}\left(f\left(t_f^i, x_f^i\right)\right)^2  \,,
\end{equation}

i.e. by evaluting the loss function based on evaluating the neural nets performance for predicting $u$, a set of $\left\{t_u^i, x_u^i, u^i\right\}_{i=1}^{N_u}$ training points (initial and boundary data) and by additionally evaluating the left hand side of the governing equation, $f$ on a set of points $\left\{t_f^i, x_f^i\right\}_{i=1}^{N_f}$.

## Solve ODEs by minimizing a combination of observations and physical (equations) 

In order to introuduce PINNS we'll start by training a neural net to represent the exponential function $y\left(x\right) = e^x$. We'll start by learning to represent the function from a set of (training) points, y. We'll then see how we can train the same network by constraining it to satisfy (or at least minimize deviation from) it's governing equation 

\begin{equation}
\frac{d \,y\left(x\right)}{dx}  - y\left(x\right) = 0  \,.
\end{equation}

 In this case we have only one independent variable, $x$, and the loss function for the regual neural network, and for the PINN simplify to:


\begin{equation}
\mathrm{loss}_\mathrm{NN} =  \mathrm{MSE}_y  = \frac{1}{N_y}\sum_{i=1}^{N_y}\left(u\left(x_y^i\right) - y_i\right)^2  \,,
\end{equation}

\begin{equation}
\mathrm{loss}_\mathrm{PINN}  = \mathrm{MSE}_u + \mathrm{MSE}_f = \frac{1}{N_{y, PINN}}\sum_{i=1}^{N_{y, PINN}}\left(u\left(x_y^i\right) - y_i\right)^2 + \frac{1}{N_f}\sum_{i=1}^{N_f}\left(f\left(x_f^i\right)\right)^2  \,,
\end{equation}

where $f = y_x - y$. Note that the number of observations $N_y$ and $N_{y, PINN}$ does not have to be the same in $\mathrm{loss}_\mathrm{NN}$ and $\mathrm{loss}_\mathrm{PINN}$, since in general we would like to use PINNS in cases where we have sparse amount of data/observations. In fact (in this example) we will only provide the PINN with one observed $y$ value. The number of points $N_f$ where $f$ is evaluated, one the other hand, is not constrained by lack of data, and we are in principle free to chose as many points as we want. But in n this example we'll compare the regular NN  and PINN in cases where $N_{y, PINN} + N_f \approx N_y$.

In both cases we'll use a single hidden layer, like the one below, however the number of neurons can be changed.


<img src="fig/neuralNet.png" width="400">

## problems
- compare accuracy (MSE) using the regular NN and PINN
- compare accuracy (MSE) using regular NN and PINN in the case of extrapolation, i.e. when training is done on a limited set of data, and predictions are obtained outside this area.
- modify the code to represent y(x) = cos(x), in which f = y + d^2y/dx^2
- modify the code to represent y(x) = e^x * cos(x), in which f: y - dy/y_x + 0.5*d^2y/dx^2
- In the latter two cases, try to increase the domain bounds. Also try with and without feeding the NN with scaled (normalized between 0 and 1) inputs. For the PINN you may need to feed it with a few more observations

## Regular neural network

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#==========================================================================================#
# Set parampeters (hyperameters) and compute a training data set for the desired function  #
#==========================================================================================#

N = 10001 # Number of training points
N_train = 10 # Number of points to train on per training attempt
N_neurons = 5 # Number of neurons in the hidden layer
afunc = tf.nn.sigmoid # Set activation function, other functions .relu, .sigmoid, .tanh, .elu

learning_rate = 0.1
epochs = 5000 # Number of epochs


#Set x-domain boundaries
x_start = 0.
x_end = 2.

#ub = np.array([[x_end]]) #upper bound
x_data = np.linspace(x_start, x_end, N)
#===================================================
# Specify what function you wish to train for
#===================================================
y_data = np.exp(x_data)


In [ ]:
x_data = x_data[:, np.newaxis] # turn 1D array into 2D array of shape (N, 1)
y_data = y_data[:, np.newaxis] # turn 1D array into 2D array of shape (N, 1)

constrainUpper = 1 # set smaller than one to test extrapolation
idx = np.random.choice(int(x_data.shape[0]*constrainUpper), N_train, replace=False)
x_data_train = x_data[idx]
y_data_train = y_data[idx]

In [ ]:
#===================================================
# set up placeholder for inputs and outputs
#===================================================
x = tf.placeholder(tf.float32, shape=[None, x_data.shape[1]], name='x')
y = tf.placeholder(tf.float32, shape=[None, y_data.shape[1]], name='y')
#===================================================
# declare weights and biases input --> hidden layer
#===================================================
W1 = tf.Variable(tf.random_normal([1, N_neurons], stddev=0.5), name='W1')
b1 = tf.Variable(tf.random_normal([N_neurons]), name='b1')
#===================================================
# declare weights and biases of hidden --> output layer
#===================================================
W2 = tf.Variable(tf.random_normal([N_neurons, 1], stddev=0.5), name='W2')
b2 = tf.Variable(tf.random_normal([1]), name='b2')
#===================================================
# declare output of NN
#===================================================
a1 = afunc(tf.add(tf.matmul(x, W1), b1)) # activation of hidden layer
y_NN = tf.add(tf.matmul(a1, W2), b2) # computational graph for the neural network

In [ ]:
#===================================================
# plot y_pred before training
#===================================================
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) # initialize variables
    y_pred_init = sess.run(y_NN, feed_dict = {x:x_data})
plt.figure()
plt.plot(x_data.flatten(), y_data.flatten())
plt.plot(x_data_train.flatten(), y_data_train.flatten(), 'o')
plt.plot(x_data.flatten(), y_pred_init.flatten())

In [ ]:
#===================================================
# train the model using regular NN
#===================================================
loss = tf.reduce_mean(tf.square(y - y_NN))
optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
print_every_N_batch = 1000
with tf.Session() as sess:
    sess.run(init) # initialize variables
    for epoch in range(epochs):
        avg_cost = 0
        _, c = sess.run([optimiser, loss], 
                     feed_dict={x: x_data_train, y: y_data_train})
        avg_cost += c
        if epoch % print_every_N_batch == 0:
            print("Epoch:", (epoch + 1), "cost =", "{:.6f}".format(avg_cost))

    y_pred = sess.run(y_NN, feed_dict = {x:x_data, y:y_data})
    loss_pred = sess.run(loss, feed_dict = {x:x_data, y:y_data})
    plt.figure()
    plt.plot(x_data.flatten(), y_data.flatten(), 'o')
    plt.plot(x_data.flatten(), y_pred.flatten(),'r--')
    plt.plot(x_data_train.flatten(), y_data_train.flatten(), 'bo')
    plt.legend(["y", "y_NN", "y_train"])
    print("MSE_u on all data (N: {}): ".format(x_data.shape[0]), loss_pred)

## PINN

In [ ]:
#===================================================
# plot y_pred before training
#===================================================
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) # initialize variables
    y_pred_init = sess.run(y_NN, feed_dict = {x:x_data})
plt.figure()
plt.plot(x_data.flatten(), y_data.flatten())
plt.plot(x_data_train.flatten(), y_data_train.flatten(), 'o')

plt.plot(x_data.flatten(), y_pred_init.flatten())

In [ ]:
#===================================================
# train the model using PINNS
#===================================================

x_f = tf.placeholder(tf.float32, shape=[None, x_data.shape[1]], name='x_f')
a1_PINNS = afunc(tf.add(tf.matmul(x_f, W1), b1))
y_NN_PINNS = tf.add(tf.matmul(a1_PINNS, W2), b2)
# note that dy_dx_NN, y_NN_PINNS and y_NN share the same weights and biases
dy_dx_NN = tf.gradients(y_NN_PINNS, x_f)[0]


f =  dy_dx_NN - y_NN_PINNS

MSE_u = tf.reduce_mean(tf.square(y - y_NN))
MSE_f = tf.reduce_mean(tf.square(f))
loss_PINNS = MSE_u + MSE_f
optimiser_PINNS = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss_PINNS)

# we only use one training point to minimize MSEu
x_data_train_PINNS = x_data_train[0:1,0:1] 
y_data_train_PINNS = y_data_train[0:1,0:1]


print_every_N_batch = 1000
with tf.Session() as sess:
    sess.run(init) # initialize variables
    for epoch in range(epochs):
        avg_cost = 0
        _, c = sess.run([optimiser_PINNS, loss_PINNS ], 
                     feed_dict={x: x_data_train_PINNS, y: y_data_train_PINNS, x_f: x_data_train})
        avg_cost += c
        MSE_u_value, MSE_f_value = sess.run([MSE_u, MSE_f], 
                     feed_dict={x: x_data_train_PINNS, y: y_data_train_PINNS, x_f: x_data_train})
        if epoch % print_every_N_batch == 0:
            print("Epoch:", (epoch + 1), "cost =", "{:.6f}".format(avg_cost), "MSE_u =", "{:.6f}".format(MSE_u_value), "MSE_f =", "{:.6f}".format(MSE_f_value))

    y_pred = sess.run(y_NN, feed_dict = {x:x_data, y:y_data})
    loss_pred = sess.run(loss, feed_dict = {x:x_data, y:y_data, x_f:x_data})
    plt.figure()
    plt.plot(x_data.flatten(), y_data.flatten(), 'o')
    plt.plot(x_data.flatten(), y_pred.flatten(),'r--')
    plt.plot(x_data_train.flatten(), y_data_train.flatten(), 'bo')
    plt.plot(x_data_train_PINNS.flatten(), y_data_train_PINNS.flatten(), 'go')
    plt.legend(["y", "y_NN", "y_train(x_f)", "y_train"])
    print("MSE_u on all data (N: {}): ".format(x_data.shape[0]), loss_pred)


## Sympy cells to calculate f for some simple functions

In [ ]:
import sympy as sp

x_sp = sp.Symbol('x')
y_sp = sp.cos(x_sp)
dy_sp_dx = sp.diff(y_sp, x_sp)
dy_sp_dx2 = sp.diff(dy_sp_dx, x_sp)
#y_sp, dy_sp_dx, dy_sp_dx2
y_sp + dy_sp_dx2

In [ ]:
x_sp = sp.Symbol('x')
y_sp = sp.cos(x_sp)*sp.exp(x_sp)
dy_sp_dx = sp.diff(y_sp, x_sp)
dy_sp_dx2 = sp.diff(dy_sp_dx, x_sp)
#y_sp, dy_sp_dx, dy_sp_dx2
y_sp - dy_sp_dx + 0.5*dy_sp_dx2